In [1]:
import numpy as np
import cv2
import sys

In [ ]:
# readNet(model[, config[, framework]]) -> retval
# net = cv2.dnn.readNet

# blobFromImage(image[, scalefactor[, size[, mean[, swapRB[, crop[, ddepth]]]]]]) -> retval
# blob = cv2.dnn.blobFromImage

# net.setInput(blob)
# prob = net.forward()

########### googLeNet 영상인식
# 입력크기: (224, 224)
# 컬러: BGR
# 밝기평균값: (104, 117, 123)

In [36]:
import os

file_lists = os.listdir('./googlenet_old/fig/')
print(file_lists)

file_names = []
for i in file_lists:
    file_name = './googlenet_old/fig/' + i
    file_names.append(file_name)

# filename = './googlenet_old/fig/apple1.png'

img = cv2.imread(filename)

if img is None:
    print('image read failed')
    sys.exit()
    
### dnn Model################################################################
# model = './googlenet/bvlc_googlenet.caffemodel'
# config = './googlenet/deploy.prototxt' 

model = './googlenet/googlenet-3.onnx'
config = '' 

net = cv2.dnn.readNet(model, config)

if net.empty():
    print('network load failed')
    sys.exit()

########## 클래스 이름 불러오기

classNames = []
with open('./googlenet/classification_classes_ILSVRC2012.txt', 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')

idx = 0
while True:
    img = cv2.imread(file_names[idx])
    # print(classNames[100])
    # blobFromImage(image[, scalefactor[, size[, mean[, swapRB[, crop[, ddepth]]]]]]) -> retval
    blob = cv2.dnn.blobFromImage(img, 1, (224, 224), (104, 117, 123),
                                swapRB = False)
    net.setInput(blob)
    prob = net.forward()
    ########################################################################

    # print(prob.shape)
    # print(type(prob))

    out = prob.flatten()
    # print(out.shape)
    classid = np.argmax(out)
    print(classid)
    confidence = out[classid]
    print(confidence)
    category = classNames[classid]
    print(category)

    text = f'{classNames[classid]} ({confidence*100:4.3f})%'

    cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_COMPLEX, 
                0.8, (0, 0, 255), 2, cv2.LINE_AA)

    cv2.imshow('img', img)
    
    if cv2.waitKey(3000) == 27:
        break
    
    idx += 1
    if idx >= len(files_names):
        idx = 0

cv2.destroyAllWindows()

['apple1.png', 'apple2.png', 'beagle.jpg', 'fish.jpg', 'pineapple.jpg', 'scooter.jpg', 'space_shuttle.jpg']
957
0.9361577
pomegranate
948
0.99973625
Granny Smith
162
0.78353864
beagle
393
0.98700094
anemone fish
953
0.99994934
pineapple, ananas
670
0.59207493
motor scooter, scooter
812
0.99993455
space shuttle
957
0.9361577
pomegranate
948
0.99973625
Granny Smith
162
0.78353864
beagle
393
0.98700094
anemone fish
953
0.99994934
pineapple, ananas
670
0.59207493
motor scooter, scooter
812
0.99993455
space shuttle
957
0.9361577
pomegranate
948
0.99973625
Granny Smith
162
0.78353864
beagle
393
0.98700094
anemone fish
953
0.99994934
pineapple, ananas
670
0.59207493
motor scooter, scooter


In [45]:
import numpy as np
import sys
import cv2

img = cv2.imread('./opencv_face_detector_old/fig/sunglass.png')

## Tensorflow
model = './face_detector/opencv_face_detector_uint8.pb'
config = './face_detector/opencv_face_detector.pbtxt'

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('Video open failed')
    cap.release()
    sys.exit()

face_net = cv2.dnn.readNet(model, config)

if face_net.empty():
    print('net load failed')
    sys.exit()
    
while True:
    ret, img = cap.read()
    if not ret:
        print('Video frame failed')
        break
        
    blob = cv2.dnn.blobFromImage(img, 1, (300, 300), (104, 177, 123),
                                swapRB = False)
    face_net.setInput(blob)
    out = face_net.forward()

    detect = out[0, 0, :, :]
    # print(detect.shape)

    h, w = img.shape[:2]
    threshold = 0.5
    for i in range(detect.shape[0]):
        confidence = detect[i, 2]

        if confidence > threshold:
            x1 = int(detect[i,3]*w)
            y1 = int(detect[i,4]*h)
            x2 = int(detect[i,5]*w)
            y2 = int(detect[i,6]*h)

            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
    #         text = f'Face: {confidence*100:4.2f}%'
            text = 'Face: {}%'.format(round(confidence*100, 2))
            cv2.putText(img, text, (x1, y1-3), cv2.FONT_HERSHEY_COMPLEX,
                       1, (255, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow('image', img)
    
    if cv2.waitKey(20) ==  27:
        break

cap.release()
cv2.destroyAllWindows()